# Ortalama Kayma (MeanShift) Algoritması

In [1]:
import cv2
import numpy as np

In [2]:
cap=cv2.VideoCapture(0)
ret,frame=cap.read()
if ret==False:
    print("okuma hatasi")
face_cascade=cv2.CascadeClassifier("input/Haar cascade/4.1 frontalface.xml.xml")
face_rects=face_cascade.detectMultiScale(frame)

# buraya kadarı benim "GoruntuIsleme/1.Egitim/14." de yaptığım gibi ama buradan sonra bu eğitimde farklı yol izliyor
(face_x,face_y,w,h)=tuple(face_rects[0])
track_winodw=(face_x,face_y,w,h) # MeanShiftalgoritması girdisi
# region of interest
roi=frame[face_y:face_y+h,face_x:face_x+w] # roi=face
hsv_roi=cv2.cvtColor(roi,cv2.COLOR_BGR2HSV)
roi_hist = cv2.calcHist([hsv_roi], [0], None, [180], [0, 180]) # takip için histogram gerekli
cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)

# takip için gerekli durdurma kriterleri
# count = hesaplanacak gerekli öge sayısı
# eps=değişiklik
term_crit=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,5,1)

while True:
    ret, frame = cap.read()
    # frame=cv2.flip(frame,1) bunu uygulayınca takip edemedi
    if ret:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        # histogramı bir görüntüde bulmak için kullanıyoruz
        # piksel karşılaştırma
        dst = cv2.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)


        ret,track_winodw=cv2.meanShift(dst,track_winodw,term_crit)
        x,y,w,h=track_winodw
        img2=cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),5)
        cv2.imshow("yuz",img2)
        if cv2.waitKey(1)&0xff==ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/ibrahim/.local/lib/python3.10/site-packages/cv2/qt/plugins"


## Tüm kod ayrıntılı açıklama :
Elbette, bu kod parçası, bir video akışındaki yüzü tespit etmek ve MeanShift algoritması kullanarak yüzü takip etmek için OpenCV kütüphanesini kullanmaktadır. Kodun her bir parçasını ayrıntılı olarak açıklayacağım:

### Kodun İlk Kısmı: Yüz Tespiti

```python
cap = cv2.VideoCapture(0)
ret, frame = cap.read()
if ret == False:
    print("okuma hatasi")
```

- `cap = cv2.VideoCapture(0)`: Bilgisayarınızın varsayılan web kamerasını (cihaz indeksi 0) kullanarak bir video yakalama nesnesi oluşturur.
- `ret, frame = cap.read()`: Kameradan bir kare (frame) okur. `ret` değeri, okumanın başarılı olup olmadığını belirtir. `frame` ise okunan görüntüdür.
- `if ret == False:`: Eğer kare okunamazsa, "okuma hatasi" mesajını yazdırır.

```python
face_cascade = cv2.CascadeClassifier("input/Haar cascade/4.1 frontalface.xml.xml")
face_rects = face_cascade.detectMultiScale(frame)
```

- `face_cascade = cv2.CascadeClassifier("input/Haar cascade/4.1 frontalface.xml.xml")`: Haar yüz tanıma kaskadını yükler. Bu dosya, önceden eğitilmiş bir yüz algılama modelini içerir.
- `face_rects = face_cascade.detectMultiScale(frame)`: Yüzleri `frame` içinde tespit eder. Bu fonksiyon, tespit edilen yüzlerin koordinatlarını içeren bir liste döner.

### Kodun İkinci Kısmı: MeanShift Algoritması İçin Hazırlık

```python
(face_x, face_y, w, h) = tuple(face_rects[0])
track_winodw = (face_x, face_y, w, h)  # MeanShift algoritması girdisi
```

- `(face_x, face_y, w, h) = tuple(face_rects[0])`: Tespit edilen ilk yüzün koordinatlarını alır.
- `track_winodw = (face_x, face_y, w, h)`: MeanShift algoritması için izleme penceresini tanımlar.

```python
roi = frame[face_y:face_y+h, face_x:face_x+w]  # roi=face
hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
roi_hist = cv2.calcHist([hsv_roi], [0], None, [180], [0, 180])  # takip için histogram gerekli
cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)
```

- `roi = frame[face_y:face_y+h, face_x:face_x+w]`: Yüz bölgesini (Region of Interest - ROI) seçer.
- `hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)`: ROI'yi HSV renk uzayına dönüştürür. HSV (Hue, Saturation, Value) renk uzayı, renk takibi için daha uygundur.
- `roi_hist = cv2.calcHist([hsv_roi], [0], None, [180], [0, 180])`: HSV ROI'nin histogramını hesaplar. Bu histogram, takip sırasında kullanılır.
- `cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)`: Histogramı normalleştirir. Histogram değerlerini 0 ile 255 arasında ölçeklendirir.

### Kodun Üçüncü Kısmı: İzleme

```python
term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 5, 1)
```

- `term_crit`: MeanShift algoritması için durdurma kriterlerini tanımlar. `cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT` ile her iki kriter de kullanılacaktır: belirli bir sayıdaki iterasyon (`count`) ve pozisyon değişim miktarı (`eps`).

```python
while True:
    ret, frame = cap.read()
    if ret:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        dst = cv2.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)

        ret, track_winodw = cv2.meanShift(dst, track_winodw, term_crit)
        x, y, w, h = track_winodw
        img2 = cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 5)
        cv2.imshow("yuz", img2)
        if cv2.waitKey(1) & 0xff == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()
```

- `while True`: Sonsuz bir döngü başlatır.
- `ret, frame = cap.read()`: Kameradan bir kare okur.
- `if ret:`: Eğer kare okunmuşsa devam eder.
    - `hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)`: Mevcut kareyi HSV renk uzayına dönüştürür.
    - `dst = cv2.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)`: Mevcut karenin histogramına dayanarak, ROI'nin olası konumlarını belirler.
    - `ret, track_winodw = cv2.meanShift(dst, track_winodw, term_crit)`: MeanShift algoritmasını kullanarak yeni izleme penceresini hesaplar.
    - `x, y, w, h = track_winodw`: Yeni izleme penceresinin koordinatlarını alır.
    - `img2 = cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 5)`: İzleme penceresini kırmızı bir dikdörtgenle çizer.
    - `cv2.imshow("yuz", img2)`: Çerçeveyi ekranda gösterir.
    - `if cv2.waitKey(1) & 0xff == ord('q')`: 'q' tuşuna basıldığında döngüyü kırar.
- `cap.release()`: Kamerayı serbest bırakır.
- `cv2.destroyAllWindows()`: Tüm OpenCV pencerelerini kapatır.
